## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-08-14-51-00 +0000,wsj,Milei’s Election Setback Rattles Markets in Ar...,https://www.wsj.com/world/americas/mileis-elec...
1,2025-09-08-14-44-22 +0000,nypost,Hurricane Kiko rolls closer to Hawaii as resid...,https://nypost.com/2025/09/08/us-news/hurrican...
2,2025-09-08-14-44-01 +0000,nyt,At Least 12 Are Killed in Nepal in Protests Ov...,https://www.nytimes.com/2025/09/08/world/asia/...
3,2025-09-08-14-43-12 +0000,bbc,Jaguar Land Rover extends shutdown after cyber...,https://www.bbc.com/news/articles/c15k9gjg05go...
4,2025-09-08-14-39-27 +0000,nypost,Curtis Sliwa’s wife Nancy promotes him for NYC...,https://nypost.com/2025/09/08/us-news/curtis-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
97,trump,23
65,new,13
30,attack,11
389,raid,11
628,ukraine,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
260,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...,80
56,2025-09-08-11-52-35 +0000,cbc,Israel vows 'hurricane' of strikes on Gaza to ...,https://www.cbc.ca/news/world/israel-gaza-city...,62
186,2025-09-07-22-34-59 +0000,latimes,"South Korea, irked at U.S. raid at Hyundai pla...",https://www.latimes.com/business/story/2025-09...,58
28,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...,58
168,2025-09-08-00-12-10 +0000,wapo,South Korea announces deal to repatriate worke...,https://www.washingtonpost.com/world/2025/09/0...,58


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
260,80,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...
186,58,2025-09-07-22-34-59 +0000,latimes,"South Korea, irked at U.S. raid at Hyundai pla...",https://www.latimes.com/business/story/2025-09...
144,47,2025-09-08-03-34-58 +0000,nypost,Father on run with 3 kids in wilderness for 4 ...,https://nypost.com/2025/09/07/world-news/fugit...
57,43,2025-09-08-11-46-03 +0000,nypost,Israel threatens to destroy Gaza City in a ‘mi...,https://nypost.com/2025/09/08/world-news/israe...
20,39,2025-09-08-14-06-44 +0000,nypost,Police in Nepal open fire on protesters outsid...,https://nypost.com/2025/09/08/world-news/polic...
145,39,2025-09-08-03-32-23 +0000,bbc,What it was like inside court as mushroom murd...,https://www.bbc.com/news/videos/c15k4y5nwz7o?a...
28,32,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...
132,32,2025-09-08-05-17-27 +0000,nypost,Elderly California jewelry store owner assault...,https://nypost.com/2025/09/08/us-news/californ...
197,27,2025-09-07-20-56-17 +0000,nypost,Trump goes off on reporter for asking if he’s ...,https://nypost.com/2025/09/07/us-news/trump-go...
95,26,2025-09-08-09-30-00 +0000,wsj,The U.S. labor market would be barely growing ...,https://www.wsj.com/economy/jobs/healthcare-jo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
